## ✅ 실습 주제: LLaMA3 기반 기술 문서 요약 자동화

### 🎯 학습 목표

- 기술 문서를 요약하는 목적과 활용 사례 이해

- 최신 LLM인 LLaMA3를 이용한 요약 자동화 방법 실습

- 프롬프트 엔지니어링과 미세조정(fine-tuning) 방식 비교

- 실제 산업에서 활용 가능한 자동화 파이프라인 체험

## 1️⃣ 실습 데이터셋 구성

### 📄 기술 문서 예시

- 제품 사양서 (예: 전자제품, 차량 부품)

- 설치 매뉴얼

- API 문서 요약

### 🔧 JSONL 구조 예시

In [ ]:
{"instruction": "다음 기술 문서를 요약해 주세요.",
 "input": "제품은 5V 전원을 사용하며, ESP32 마이크로컨트롤러를 기반으로 동작합니다. 무선 통신은 BLE와 WiFi를 모두 지원하며...",
 "output": "• 5V 전원 기반\n• ESP32 칩셋 사용\n• BLE 및 WiFi 통신 지원"
}

👉 데이터는 수작업 레이블링 혹은 기존 문서를 이용한 반자동 생성 가능

👉 Colab에서 pandas, json 사용해 전처리

## 2️⃣ 실습 환경 준비 (Colab 기준)

### 🔗 사전 준비

In [ ]:
!pip install transformers accelerate bitsandbytes
!pip install sentencepiece

###📦 모델 로딩 (LLaMA3 기반)

### 1️⃣ Hugging Face 계정 생성 및 로그인

1. https://huggingface.co/join 접속 후 계정 생성 또는 로그인

2. 아래 링크에서 LLaMA 3 모델 접근 신청 (수동 승인 필요함)

  - 🔗 Meta-LLaMA/Meta-Llama-3-8B-Instruct

  - "Access repository" 버튼 클릭 → Use agreement 체크 후 신청

###2️⃣ Hugging Face 토큰 발급 및 설정

🔑 토큰 발급

- https://huggingface.co/settings/tokens 접속 → "New token" 클릭

- 권한: Read

- 발급된 토큰 복사

🧪 Colab에서 로그인 (토큰 설정)


```
from huggingface_hub import login
login(token="hf_...")  # 위에서 발급받은 토큰 입력
```




In [ ]:
from huggingface_hub import login
login(token="여러분의 토큰을 넣으시면 됩니다.")

### 3️⃣ 모델 로딩 시도

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "meta-llama/Meta-Llama-3-8B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", load_in_8bit=True)

##3️⃣ 프롬프트 기반 요약 실습

###🧪 단일 문서 요약 실습

In [ ]:
doc = """
본 차량의 전방 카메라는 고정밀 운전자 지원 시스템(ADAS)에 핵심적으로 사용되는 장치입니다. 8MP 해상도를 지원하며, Sony IMX490 센서를 채택하여 야간 및 역광 상황에서도 뛰어난 시인성을 제공합니다. HDR(High Dynamic Range) 기능을 통해 터널이나 갑작스러운 조도 변화 상황에서도 안정적인 이미지를 생성할 수 있습니다.
카메라는 120도의 시야각(Field of View)을 제공하며, 최대 60fps로 실시간 영상 스트리밍이 가능합니다. ISP(이미지 신호 프로세서)는 Qualcomm Vision SDK를 기반으로 동작하며, CNN 기반 사물 인식 기능이 내장되어 있습니다.
작동 전압은 5V이며 소비 전력은 최대 2.5W입니다. 이 카메라는 차량의 CAN 네트워크와 연동되어 ADAS ECU로 실시간 데이터를 전송합니다. EMC 인증을 받았으며, 동작 온도 범위는 -40°C ~ 85°C입니다. IP69K 방수/방진 등급을 지원하여 외부 장착에도 적합합니다.
"""
prompt = f"다음 기술 문서를 요약해 주세요:\n\n{doc}\n\n요약:"
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

summary_ids = model.generate(**inputs, max_new_tokens=150)
print(tokenizer.decode(summary_ids[0], skip_special_tokens=True))


## 🤖 고객 서비스 챗봇 개발 실습 (LLaMA3 기반)
### 프롬프트 기반 LLaMA3 응답 생성

In [ ]:
def llama_answer(user_input):
    prompt = f"""당신은 고객센터 상담 챗봇입니다.
다음 사용자 질문에 대해 간결하고 정확하게 응답하세요.
그리고 한국말로 대답하세요

고객: {user_input}
챗봇:"""

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(**inputs, max_new_tokens=100)
    result = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # 응답 부분만 추출
    return result.split("챗봇:")[-1].strip()

🔎 예시 실행

In [ ]:
user = input("고객 질문: ")
print("🤖 챗봇:", llama_answer(user))

### 🧭 단계별 프롬프트 설계 프레임워크

| 단계  | 핵심 목표           | 학습 내용                     | 예시                            |
| --- | --------------- | ------------------------- | ----------------------------- |
| 1단계 | **명령어 주기**      | 단순 지시로 시작                 | "설명해줘", "요약해줘"                |
| 2단계 | **컨텍스트 제공**     | 배경/조건 부여                  | "학생을 대상으로", "초보자 기준으로"        |
| 3단계 | **출력 형식 명시**    | 리스트, 표, JSON 등 구조화된 결과 요청 | "표 형식으로", "코드로"               |
| 4단계 | **역할 지정**       | AI에게 역할을 부여해 스타일/톤 제어     | "너는 수학 선생님이야"                 |
| 5단계 | **Few-shot 학습** | 예시 입력-출력을 미리 제공           | "입력: ..., 출력: ..." 반복 후 실제 질문 |


### 🚗 자동차 도메인 기반 프롬프트 설계 5단계 실습

###🔧 준비 코드 (Hugging Face LLaMA3 사용)

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

def ask_llama(prompt):
    prompt += "한국말로 대답하세요"
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(**inputs, max_new_tokens=200)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)


###🚘 1단계: 단순 지시

In [ ]:
prompt = "현대자동차 아이오닉5의 특징을 설명해줘."
print(ask_llama(prompt))

###🚘 2단계: 대상자 기준 조건 추가

In [ ]:
prompt = "자동차에 대해 잘 모르는 사람에게 아이오닉5의 주요 특징을 쉽게 설명해줘."
print(ask_llama(prompt))

###🚘 3단계: 출력 형식 지정

In [ ]:
prompt = """아이오닉5의 장단점을 표 형식으로 정리해줘.

[형식 예시]
장점 | 단점
---- | ----
...  | ...
"""
print(ask_llama(prompt))


###🚘 4단계: 역할 부여

In [ ]:
prompt = """너는 전기차 딜러야. 고객에게 아이오닉5를 추천하는 세일즈 멘트를 작성해줘."""
print(ask_llama(prompt))

### 🚘 5단계: Few-shot 예시 기반 프롬프트

In [ ]:
prompt = """
다음은 자동차 상담 Q&A입니다.

Q: '전기차는 겨울에 주행거리가 줄어드나요?'
A: '네, 추운 날씨에서는 배터리 효율이 낮아져 주행거리가 줄 수 있습니다.'

Q: '아이오닉5 충전 시간은 얼마나 걸리나요?'
A: '초고속 충전기 기준으로 약 18분이면 80%까지 충전됩니다.'

Q: '아이오닉5의 주행거리는 얼마인가요?'
A:
"""
print(ask_llama(prompt))


## 🚗 실습 예제: LLaMA3로 자동차 설명서 자동 생성하기

###🧾 Step 1: 차량 사양 (입력 JSON)

In [ ]:
car_spec = {
    "model": "아이오닉5",
    "battery_capacity": "77.4 kWh",
    "range": "429 km",
    "charging_time": "18분 (350kW 초고속 충전 기준)",
    "charging_types": ["완속 충전", "급속 충전", "V2L"],
    "special_features": ["배터리 히팅 시스템", "충전 예약 기능"]
}

### ✏️ Step 2: 프롬프트 생성

In [ ]:
prompt = f"""
너는 자동차 사용설명서를 작성하는 기술 작가야.
다음 차량 사양 정보를 참고해서 '배터리 및 충전' 항목을 작성해줘.

차량 모델: {car_spec['model']}
배터리 용량: {car_spec['battery_capacity']}
1회 충전 주행 거리: {car_spec['range']}
충전 시간: {car_spec['charging_time']}
지원 충전 방식: {', '.join(car_spec['charging_types'])}
특수 기능: {', '.join(car_spec['special_features'])}

[요구사항]
- 설명서 스타일로 작성할 것
- 문단 2~3개
- 기술 용어는 쉽게 설명할 것
- 한글로 대답 할것
"""

###🤖 Step 3: LLaMA3 모델 로딩 및 실행

In [ ]:
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
outputs = model.generate(**inputs, max_new_tokens=500)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))